# Determining Water Potability

**Introduction**

Water potability is critical for supporting populations, various industries and agricultural activities that rely on clean water for survival. Clean and drinkable water can be determined by careful analysis of certain characteristics of a sample. Using classification we hope to train a data set to be able to classify water as safe or not based on its characteristics. The question we will be asking is: Is this water sample potable based on the different levels and characteristics of it?
The data set we will be using is ‘Water Quality and Potability’. Each observation has a pH value, total solids dissolved, chloramines count, sulfate count, conductivity, organic carbon level, trihalomethanes, turbidity and potability. For the sake of our project we have decided to utilize all of the vectors this data set provides except for conductivity. With these values we hope to train a classification system that can provide accurate results on potability. 


**The Dataset**

In [3]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source("cleanup.R")


Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'tests.R': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [4]:
#reading the data set from the web 
url <- "https://raw.githubusercontent.com/nori-2004/data-science-group-project/main/water_potability.csv"
water <- read_csv(url)

#cleaning the data by selecting the columns we want to use
water_selected <- select(water, - Conductivity)
water_selected

Rows: 3276 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (10): ph, Hardness, Solids, Chloramines, Sulfate, Conductivity, Organic_...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ph,Hardness,Solids,Chloramines,Sulfate,Organic_carbon,Trihalomethanes,Turbidity,Potability
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NA,204.8905,20791.32,7.300212,368.5164,10.37978,86.99097,2.963135,0
3.716080,129.4229,18630.06,6.635246,NA,15.18001,56.32908,4.500656,0
8.099124,224.2363,19909.54,9.275884,NA,16.86864,66.42009,3.055934,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
9.419510,175.7626,33155.58,7.350233,NA,11.03907,69.84540,3.298875,1
5.126763,230.6038,11983.87,6.303357,NA,11.16895,77.48821,4.708658,1
7.874671,195.1023,17404.18,7.509306,NA,16.14037,78.69845,2.309149,1


In [5]:
#splitting the data into testing and training sets
water_split <- initial_split(water_selected, prop = 0.75)
water_train <- training(water_split)   
water_test <- testing(water_split)
water_train

ph,Hardness,Solids,Chloramines,Sulfate,Organic_carbon,Trihalomethanes,Turbidity,Potability
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
8.423084,153.0329,15924.80,11.101628,329.1322,12.321633,52.63131,3.958609,1
7.222998,179.5603,21314.87,5.864717,NA,11.878370,79.39401,3.966259,1
3.641630,183.9087,24752.07,5.538314,286.0596,9.034067,73.59466,3.464353,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
7.010243,162.1232,26395.25,6.622597,NA,12.27799,51.62869,3.279006,0
6.109073,191.7556,26853.95,9.064613,312.2020,15.51403,73.78968,4.881101,0
10.893485,177.7344,25702.52,7.659830,312.3993,17.47211,42.81457,3.634970,0
